# Location-Specific Identity

Let's get an instance of PostgreSQL running in Docker:

```
docker run --name postgres -p5432:5432 -e POSTGRES_PASSWORD=mypw -d postgres
```

Then we can connect to it by starting the CLI and entering this command:

```
psql -U postgres
```

Postgres has an object called a Sequence. It's entire purpose is to generate increasing numbers. Every time someone asks for a number from the sequence, they are guaranteed to get a number different -- and bigger -- than any number that anyone else has seen.

Create a sequence like this:

```
CREATE SEQUENCE id;
```

Then get the next number like this:

```
SELECT nextval('id');
```

You can also do that in a client application using a Postgres driver.
This code will install a driver via a NuGet package and then get the next ID from the sequence.

In [ ]:
#r "nuget: Npgsql, 5.0.7"

using Npgsql;

var connString = "Host=localhost;Username=postgres;Password=mypw;Database=postgres";

var conn = new NpgsqlConnection(connString);
await conn.OpenAsync();

var comm = new NpgsqlCommand("SELECT nextval('id')", conn);
var nextId = await comm.ExecuteScalarAsync();
await comm.DisposeAsync();

await conn.DisposeAsync();

nextId

1

The way that it works is that the Sequence stores a number on a hard drive. When someone asks for a number, it reads the number from the hard drive, increments it, and writes it back. It uses synchronization primitives to make sure that it only produces one number at a time. The number is stored in a location.

The reason that Postgres has Sequences is to produce identities. An application will get a number from a Sequence before inserting into a table. It will use that number as the primary key. As long as the application is playing nice, the primary key of that new row is unique.

Since the number is stored in a location -- like a hard drive or memory -- and it is used to identify a row, I call this "location-dependent identity". If an application wants to create a new object, it has to go to that location, and take a lock, before it can determine the new object's identity. This is a topological bottleneck in a distributed system.

# Location-Independent Identity

What I prefer to do instead is to use "location-independent identity". That is identity which does not depend upon a certain special storage location in order to generate something unique. Location-indepedent identity lets parts of a distributed system create new objects without having to talk to each other or wait in line. It's the first step to building a system that is resilient and scalable.

One kind of location-independent identity that comes to mind is the GUID: the Globally Unique Identifier. It also goes by the much more ambitious name of the UUID: Universally Unique Identifier. The universe is much bigger than the globe. Here's a GUID:

In [ ]:
Guid.NewGuid()

98d0625c-f480-4d9c-9337-70a2f04794a2

Any machine is capable of generating a GUID without talking to another machine or waiting in line. And it has a really good chance of being unique. That's partly to do with its structure, but mostly to do with randomness.

A GUID is expressed as hexadecimal digits: 0 through f. One digit represents 4 bits of information, as 4 bits can be set in 16 different ways. A GUID has 8+4+4+4+12 = 32 digits, giving 32*4 = 128 bits of information. This is sometimes called a measure of the _entropy_ of the identifier. (Actually, a GUID has less than that because it contains a version number that takes up some of the bits. That 4 at the beginning of the third section is very predictable.)

If you want an even better guarantee of uniqueness, then you can ask for more bits of entropy. Here's one that works pretty well. Find a random number from 0 to 2^1024-1. It takes 1024 bits to express this number. Then check to see if it is prime. If not, find a different random number. The number of primes less than N is about the square root of N, and taking the square root cuts the number of bits in half. So your 1024-bit number actually only has 512 bits of entropy.

Once you have your 1024-bit prime, find a second one. Multiply these together to get a 2048-bit number. This number will have 1024 bits of entropy. That's a lot more than the 128 (or more like 124) bits of entropy in a GUID. Here's what a number like that looks like:

In [ ]:
using System.Security.Cryptography;

var rsa = RSA.Create(2048);
byte[] bytes = rsa.ExportRSAPublicKey();
string publicKey = Convert.ToBase64String(bytes);
publicKey

MIIBCgKCAQEAtfzCjGf695r+Hz7H3XNNYZjG3By837mvJziRnZeovZgrCPf/I9bkSBAtjvxikjqD5EKxryisaqv5AncoY7fdn3LDxf5/HrdZcPV6GyTHkQAlIO8/dzVcvIzfRoP7qWROXGhBinTPNGUW65/0HItikM0s2AvaUah31XKnKq0b/WUAdAlVuy0EqhT83iBoX3HzlsaPsoHl/VDKZlgqEW2PkrrfbcUBwqZ5xFtK4TlYyvYzvzQ77qtKtq8mCgPY/jR+0sFl129jys7/Vg+nuKPro8Kk5E1WCknw1N8cCGZWQfvJkbN/AZCQDM4r/z45n4UdrIH71Lou88q3CxNN9sj4IQIDAQAB

That number is written in base 64, which has 64 digits as opposed to hexadecimal's 16. It takes 6 bits to write a number between 0 and 63. To write just the product of the two primes would take 2048/6 = 341 digits, but this representation takes 360 digits. Some of them encode other values that are used together with the 2048 bit product. The MIIBCgKCAQEA at the beginning and AQAB at the end are very predictable.

This number has some very useful properties, aside from having a lot of entropy. The two primes that were multiplied to create the large number can be kept secret. Only the person who has those secret numbers could digitally sign a document, although it only takes their product to verify that signature. And anybody with the product can encrypt a message, but only the person with the two primes can decrypt it.

# Identity of a Person

It turns out that signing documents and decrypting messages are two things that people really like to do. So we will use a 2048-bit product of two prime numbers as the location-independent identity of a person.

In [ ]:
record Person(string publicKey);

new Person(publicKey)

publicKey
MIIBCgKCAQEAtfzCjGf695r+Hz7H3XNNYZjG3By837mvJziRnZeovZgrCPf/I9bkSBAtjvxikjqD5EKxryisaqv5AncoY7fdn3LDxf5/HrdZcPV6GyTHkQAlIO8/dzVcvIzfRoP7qWROXGhBinTPNGUW65/0HItikM0s2AvaUah31XKnKq0b/WUAdAlVuy0EqhT83iBoX3HzlsaPsoHl/VDKZlgqEW2PkrrfbcUBwqZ5xFtK4TlYyvYzvzQ77qtKtq8mCgPY/jR+0sFl129jys7/Vg+nuKPro8Kk5E1WCknw1N8cCGZWQfvJkbN/AZCQDM4r/z45n4UdrIH71Lou88q3CxNN9sj4IQIDAQAB


Here we're taking advantage of a new C# construct called a _record_. A record is a class that is immutable by default. That means that the fields can't change. By deciding that the fields can't change, C# can implement Equals and GetHashCode based on those fields. Two people with the same public key must be the same person.

In [ ]:
new Person(publicKey).Equals(new Person(publicKey))

True

# Simulating Mutability

A person has a name.
But we don't put that person's name in the Person record.
The reason is that we want to preserve the immutability of records.
So instead, we will define a new record that holds the person's name.

In [ ]:
record PersonName(Person person, string value);

Now we can keep records of people and their names.

In [ ]:
var recordsOfTypePerson = new HashSet<Person>();
var recordsOfTypePersonName = new HashSet<PersonName>();

var george = new Person(publicKey);
recordsOfTypePerson.Add(george);
recordsOfTypePersonName.Add(new PersonName(george, "George"));

To find a person's name, we can just search the collection of records.

In [ ]:
var georgesName =
    from personName in recordsOfTypePersonName
    where personName.person.Equals(george)
    select personName;

georgesName.Single().value

George